<span style="font-size: 30px;">User Feedback</span>

In [1]:
import openai
def callGPTAdaptive(messages):
    openai.api_type = "azure"
    openai.api_base = "REMOVED"
    openai.api_key = 'REMOVED'
    openai.api_version = "2023-05-15"
    response = openai.ChatCompletion.create(
        engine="GPT4",
        messages=messages
        
    )
   
    return (response['choices'][0]['message']['content'])


In [3]:
convo = [
"Friend 1: hey, did you hear? it's Jack's birthday next week!",
"Friend 2:yeap, i know!", 
"Friend 1:well, I thought we could plan a surprise party for him. what do you think?",
"Friend 2:we could do that but parties can get boring ", #1
"Friend 1:we can do something else then. like a movie night at the theater? ",
"Friend 2:we could do that yes!!",
"Friend 1:awesome! we can pick a movie that everyone likes. Are you into horror films?",
"Friend 2: horror can be quite appealing for those who like a bit of a fright. It's not my cup of tea, though.", #2
"Friend 1:LOL. sometimes a good horror flick can be a fun experience.",
"Friend 2:if jack likes it, i am ok with it i guess!",
"Friend 1:by the way, have you thought about what kind of birthday present we should get him?",
"Friend 2: is it possible that we get a gift card instead?", #3
"Friend 1:it might be nice to show him that we put some thought into it. Plus, it's his birthday, and he's our friend.",
"Friend 2:yeah, you are actually right. we can go to the store tomorrow to see what we can get.",
"Friend 1:We can split costs. What is your budget?",
"Friend 2:hmm I am not sure. I wasn't planning on spending too much.", #4
"Friend 1:okay sure… There is a store at a 20 min walk from the office. We can go there.",
"Friend 2:alright, I'll put on my joggers then haha",
"Friend 1:LOL! Should we invite others too?",
"Friend 2:i am not too sure about that", #5
"Friend 1:why not?",
"Friend 2:I find coordinating with others a bit of a challenge, but if you can do it, then sure.", #6
"Friend 1:okay! lets talk about this later. see you at the office tomorrow then.",
"Friend 2:See you!"]

In [18]:
sampleConvo = "Friend1: Hey! Do you want to go for day trip anytime soon?\nFriend2: Well, I was thinking, how about a trip to Gloucester, Massachusetts this weekend?"
sampleOutput = "0/10"
task = "\n\nOut of 10 points, rate how blunt was the tone/intent of the last message in the conversation above."
samplePrompt = sampleConvo + task

In [20]:
import time
def adaptiveRoutine(mode):
    responses=[]
    messages=[
        {"role":"user","content":samplePrompt},
        {"role":"assistant","content":sampleOutput},

    ]
    currentConvo=""
    lastFeedback = ""
    count=0
    for i in range(len(convo)):
        currentConvo = currentConvo + convo[i] + "\n"
        
        if((((i+1)%4==0) or (i==len(convo)-3)) and (i< (len(convo)-1))):
            
            messages.append({"role":"user","content": currentConvo + task})
            #for j in messages:
            #    print(j)
                
            response = callGPTAdaptive(messages)
            time.sleep(2)
            #print(messages,'\n\n')
            count+=1
            print("Response#" + str(count) +":\n" + response
                  +"\n")
            response = response.replace("\n","").replace(" ","")
            responses.append(response)
            messages.append({"role":"assistant","content":response})
            if(mode=="blunt"):
                lastFeedback = "The previous message was more blunt than your rating. Take this as ground truth, compare it with your previous output and adjust the sensitivity of your classification for future if needed."
                messages.append({"role":"user","content":lastFeedback})
            else:
                lastFeedback = "The previous message was less blunt than your rating. Take this as ground truth, compare it with your previous output and adjust the sensitivity of your classification for future if needed."
                messages.append({"role":"user","content":lastFeedback})
    
            # userInput = input ('Please provide feedback in terms of "y" or "n"')
            # if userInput == "y":
            #     lastFeedback = "I agree with your last answer - that message was blunt/rude. Take this and prior feedback into account to adjust the sensitivity of your classification."
            #     messages.append({"role":"user","content":lastFeedback})
            # elif userInput == "n":
            #     #lastFeedback = "I do not agree with you that the last message you classified was blunt/rude as this is a close friend of mine."
            #     #lastFeedback = "I do not agree with you that the last message you classified was blunt/rude as this is a close friend of mine. Reduce your sensitivity a little."
            #     lastFeedback = "I do not agree with your last answer - that message was not blunt as this is how these 2 friends, who are very close, talk to each other. Take this and prior feedback into account to adjust the sensitivity of your classification."
            #     messages.append({"role":"user","content":lastFeedback})
            # else:
            #     return
            

        
        

In [21]:
adaptiveRoutine("blunt")

Response#1:
6/10

Response#2:
4/10

Response#3:
2/10

Response#4:
3/10

Response#5:
5/10

Response#6:
4/10

